In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

  Using cached pydantic_core-2.14.6-cp311-none-win_amd64.whl.metadata (6.6 kB)
  Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
Using cached pydantic_core-2.14.6-cp311-none-win_amd64.whl (1.9 MB)
Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl (15.9 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geospatial 0.10.2 requires geotiff, which is not installed.
geospatial 0.10.2 requires lonboard, which is not installed.
pygis 0.6.3 requires kaleido, which is not installed.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
geocube 0.5.3 requires numpy<2,>=1.20, but you have numpy 2.0.2 which is incompatible.
keplergl 0.3.2 requires ipywidgets<8,>=7.0.0, but you have ipywidgets 8.1.2 which is incompatible.
leafmap 0.36.9 requires pystac-client>=0.8.2, but you have pystac-client 0.3.0 which is incompatible.
matplotlib 3.8.3 requires numpy<2,>=1.21, but you have numpy 2.0.2 which is incompatible.
numba 0.59.1 requ

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 466.4 kB/s eta 0:00:27
     -- ------------------------------------ 0.8/12.8 MB 633.2 kB/s eta 0:00:19
     -- ------------------------------------ 0.8/12.8 MB 633.2 kB/s eta 0:00:19
     -- ------------------------------------ 0.8/12.8 MB 633.2 kB/s eta 0:00:19
     --- ----------------------------------- 1.0/12.8 MB 578.7 kB/s eta 0:00:21
     --- ----------------------------------- 1.0/12.8 MB 578.7 kB/s eta 0:00:21
     --- ----------------------------------- 1.3/12.8 MB 559.2 kB/s eta 0:00:21
     ---

In [2]:
from typing import List, Tuple
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import spacy
from datasets import load_dataset


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start(

AttributeError: _ARRAY_API not found

In [3]:
def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the validation set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    unknown_categories = set(y_test) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the test set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    doc = nlp_pipeline(s)
    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]
    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [4]:
train_data, val_data, test_data, classes_list = load_sib200_ru()

In [5]:
print(f'Categories: {classes_list}')

print(len(train_data[0]))
print(len(train_data[1]))

print(len(val_data[0]))
print(len(val_data[1]))

print(len(test_data[0]))
print(len(test_data[1]))

Categories: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']
701
701
99
99
204
204


In [6]:
nlp = spacy.load('ru_core_news_sm')

print(train_data[0][0])
print(normalize_text(train_data[0][0], nlp))

print(val_data[0][0])
print(normalize_text(val_data[0][0], nlp))

print(test_data[0][0])
print(normalize_text(test_data[0][0], nlp))

Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным.
турция с <NUM> сторона окружить морями на запад эгейским на север чёрный и на юг средиземный
Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.
если увеличить расстояние для бег с <NUM> до <NUM> миля скорость становиться не так важный тогда как выносливость превращаться в абсолютный необходимость
Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций.
мутация вносить новый генетический вариация в тот время как отбор убирать её из набор проявляться вариация


In [7]:
class_probability = 1.0 / len(classes_list)
max_df = 1.0 - 0.2 * class_probability
print(f'Maximal document frequency of term is {max_df}.')

Maximal document frequency of term is 0.9714285714285714.


In [8]:
classifier = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1)),
    ('cls', LogisticRegression(solver='saga', max_iter=100, random_state=42))
])

In [9]:
cv = GridSearchCV(
    estimator=classifier,
    param_grid={
        'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'cls__C': [1e-1, 1, 10, 100, 1000],
        'cls__penalty': ['l1', 'l2']
    },
    scoring='f1_macro',
    cv=5,
    refit=True,
    n_jobs=-1,
    verbose=True
)

In [10]:
cv.fit([normalize_text(it, nlp) for it in train_data[0]], train_data[1])

Fitting 5 folds for each of 30 candidates, totalling 150 fits


C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_df=0.9714285714285714,
                                                        token_pattern='\\w+')),
                                       ('cls',
                                        LogisticRegression(random_state=42,
                                                           solver='saga'))]),
             n_jobs=-1,
             param_grid={'cls__C': [0.1, 1, 10, 100, 1000],
                         'cls__penalty': ['l1', 'l2'],
                         'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             scoring='f1_macro', verbose=True)

In [11]:
print('Best parameters:')
print(cv.best_params_)

print('Best F1-macro:')
print(cv.best_score_)

print(f'Vocabulary size is {len(cv.best_estimator_.named_steps["vectorizer"].vocabulary_)}.')

Best parameters:
{'cls__C': 1000, 'cls__penalty': 'l2', 'vectorizer__ngram_range': (1, 1)}
Best F1-macro:
0.6451646278967693
Vocabulary size is 4359.


In [12]:
y_pred = cv.predict([normalize_text(it, nlp) for it in val_data[0]])
print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

y_pred = cv.predict([normalize_text(it, nlp) for it in test_data[0]])
print(classification_report(y_true=test_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.80      0.44      0.57         9
         geography       0.67      0.75      0.71         8
            health       1.00      0.45      0.62        11
          politics       0.77      0.71      0.74        14
science/technology       0.62      0.80      0.70        25
            sports       0.75      0.75      0.75        12
            travel       0.57      0.65      0.60        20

          accuracy                           0.68        99
         macro avg       0.74      0.65      0.67        99
      weighted avg       0.71      0.68      0.67        99

                    precision    recall  f1-score   support

     entertainment       0.89      0.42      0.57        19
         geography       0.64      0.53      0.58        17
            health       0.47      0.41      0.44        22
          politics       0.78      0.83      0.81        30
science/technology       0.65      0